In [1]:
mode = '_mean' if False else '_all'
print(mode)

In [2]:
import pandas as pd
import numpy as np
import gc
from sklearn.pipeline import Pipeline, TransformerMixin
from os import path

In [3]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [4]:
file_dtype = {
    'weather_test' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16},
    'weather_train' : {'site_id': np.int8, 'air_temperature': np.float, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16}
}

def loadFile(name):
    return ConvertToDatetime().transform(
            pd.read_csv('../input/ashrae-energy-prediction/' + name + '.csv', dtype=file_dtype[name]))
        
weather_train = loadFile('weather_train')
weather_test = loadFile('weather_test')


In [5]:
# align weather
def weatherSiteOffsets():
    weather = pd.concat([weather_train,weather_test],ignore_index=True)
    weather['timestamp'] = pd.to_datetime(weather['timestamp'])
    weather_key = ['site_id', 'timestamp']

    temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

    # calculate ranks of hourly temperatures within date/site_id chunks
    temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')
    
    # create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
    df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

    # Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
    site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
    site_ids_offsets.index.name = 'site_id'
    return site_ids_offsets

site_time_offsets_df = weatherSiteOffsets()

def alignWeather(df):
    df['offset'] = df.site_id.map(site_time_offsets_df)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned'], df['offset']
    gc.collect()
    return df

#weather_train = alignWeather(weather_train)
#weather_test = alignWeather(weather_test)



In [6]:
if path.exists('../input/ashrae-energy-prediction/weather_train_s_radiation.pickle'):
    weather_train['s_radiation'] =  pd.read_pickle('../input/ashrae-energy-prediction/weather_train_s_radiation.pickle')
    weather_test['s_radiation'] =  pd.read_pickle('../input/ashrae-energy-prediction/weather_test_s_radiation.pickle')
    weather_train['s_radiation'] = weather_test['s_radiation'].astype(np.float16)
    weather_test['s_radiation'] = weather_test['s_radiation'].astype(np.float16)
print(weather_train.head())

   site_id           timestamp  air_temperature  cloud_coverage  \
0        0 2016-01-01 00:00:00             25.0             6.0   
1        0 2016-01-01 01:00:00             24.4             NaN   
2        0 2016-01-01 02:00:00             22.8             2.0   
3        0 2016-01-01 03:00:00             21.1             2.0   
4        0 2016-01-01 04:00:00             20.0             2.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0         20.00000                NaN              1019.5             0.0   
1         21.09375               -1.0              1020.0            70.0   
2         21.09375                0.0              1020.0             0.0   
3         20.59375                0.0              1020.0             0.0   
4         20.00000               -1.0              1020.0           250.0   

   wind_speed  s_radiation  
0    0.000000          0.0  
1    1.500000          0.0  
2    0.000000          0.0  
3    0.000000     

In [7]:
# TODO: do something with race precipitation is coded as a “-1” value
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113103#latest-664978
# Test out below
def cleanPrecipDepth(df):
    df.loc[df['precip_depth_1_hr'] == -1, 'precip_depth_1_hr'] = 0.25
    return df

#weather_train = cleanPrecipDepth(weather_train)
#weather_test = cleanPrecipDepth(weather_test)

#print(weather_train['precip_depth_1_hr'].value_counts().sort_index())

In [8]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            # TODO add in +9 as a NAN
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        del median
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [9]:
def fill_with_po3(wmatrix):
    return wmatrix.fillna(wmatrix.interpolate(method='polynomial', order=3))

def fill_with_lin(wmatrix):
    return wmatrix.fillna(wmatrix.interpolate(method='linear'))

def fill_with_mix(wmatrix):
    wmatrix = (wmatrix.fillna(wmatrix.interpolate(method='linear', limit_direction='both')) +
               wmatrix.fillna(wmatrix.interpolate(method='polynomial', order=3, limit_direction='both'))
              ) * 0.5
    # workaround: fill last NANs with neighbour
    assert wmatrix.count().min() >= len(wmatrix)-1 # only the first item is missing
    return wmatrix.fillna(wmatrix.iloc[1])         # fill with second item
    
class InterpolateTempMix(TransformerMixin):
    
    #https://www.kaggle.com/rcpeters/clean-weather-data-eda
    def transform(self, w_df, **transform_params):
        print(w_df.shape[0])
        print(w_df.head())
        new_df = None
        for col in ['air_temperature', 'dew_temperature']:
            w_df[col] = w_df[col].astype(np.float32) # 16 doesn't support unstack
            filled = fill_with_mix(w_df.pivot(index='timestamp', columns='site_id', values=col))
            filled = filled.sort_index().unstack().to_frame(col)
            filled = filled.reset_index()  
            if new_df is None:
                new_df = filled
            else:
                new_df[col] = filled[col]
            new_df[col] = new_df[col].astype(np.float16) 
            print(new_df.shape[0])
        new_df = new_df.reset_index(drop=True).merge(w_df[['site_id', 'timestamp', 'cloud_coverage', 'precip_depth_1_hr', 'wind_direction', 'wind_speed', 'sea_level_pressure']],
                           how='left', on=['site_id', 'timestamp'])
        return new_df

    def fit(self, X, y=None, **fit_params):
        return self
#print(weather_train.head(20))
#print(InterpolateTempMix().transform(weather_train.s)

In [10]:
class RelativeHumidity(TransformerMixin):
        
    def transform(self, df, **transform_params):
        exp   = pd.np.exp
        df['relative_humidity'] = 100*(exp((17.625*df['dew_temperature'])/(243.04+df['dew_temperature'])) / exp((17.625*df['air_temperature'])/(243.04+df['air_temperature'])))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
                                             
print(RelativeHumidity().transform(weather_train.sample(20)))

        site_id           timestamp  air_temperature  cloud_coverage  \
86102         9 2016-10-30 05:00:00             21.1             0.0   
1183          0 2016-02-19 07:00:00             12.2             0.0   
16812         1 2016-12-01 09:00:00             -1.3             0.0   
36467         4 2016-02-26 14:00:00             11.1             NaN   
22728         2 2016-08-03 21:00:00             36.1             NaN   
50748         5 2016-10-13 17:00:00             11.0             NaN   
117442       13 2016-06-02 11:00:00             13.9             NaN   
61995         7 2016-01-26 23:00:00              4.0             NaN   
37787         4 2016-04-21 14:00:00             16.1             NaN   
93222        10 2016-08-21 23:00:00             31.7             0.0   
32263         3 2016-09-04 08:00:00             21.7             NaN   
26980         3 2016-01-28 02:00:00              1.7             4.0   
67026         7 2016-08-26 18:00:00             26.5            

In [11]:
# TODO: try both for direction
class ImputeWeather(TransformerMixin):

    def __init__(self, method:str='linear', gap_limit:int=None, limit_direction:str='both'):
        self._method = method
        self._gap_limit = gap_limit
        self._limit_direction = limit_direction
        
    def transform(self, weather_df, **transform_params):
        grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=self._method, limit=self._gap_limit, limit_direction=self._limit_direction))
        if 'cloud_coverage' in grouped_weather_df.columns:
            grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
        grouped_weather_df.reset_index(inplace=True)
        weather_df = grouped_weather_df.drop(['index'], axis=1)
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
    
print(weather_train.head(20))
print(ImputeWeather().transform(weather_test))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2016-01-01 00:00:00             25.0             6.0   
1         0 2016-01-01 01:00:00             24.4             NaN   
2         0 2016-01-01 02:00:00             22.8             2.0   
3         0 2016-01-01 03:00:00             21.1             2.0   
4         0 2016-01-01 04:00:00             20.0             2.0   
5         0 2016-01-01 05:00:00             19.4             NaN   
6         0 2016-01-01 06:00:00             21.1             6.0   
7         0 2016-01-01 07:00:00             21.1             NaN   
8         0 2016-01-01 08:00:00             20.6             NaN   
9         0 2016-01-01 09:00:00             21.1             NaN   
10        0 2016-01-01 10:00:00             21.1             NaN   
11        0 2016-01-01 11:00:00             20.6             NaN   
12        0 2016-01-01 12:00:00             18.9             6.0   
13        0 2016-01-01 13:00:00             20.0

In [12]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [13]:
# TODO: rename to rolling
class AddWeatherRolls(TransformerMixin):
    
    def __init__(self, window, center=False):
        self._window = window
        self._center = center
        
    def transform(self, weather_df, **transform_params):
        group_df = weather_df.groupby(['site_id'])
        cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
        if 's_radiation' in weather_df.columns:
            cols.append('s_radiation')
        if 'relative_humidity' in weather_df.columns:
            cols.append('relative_humidity')
        rolled = group_df[cols].rolling(window=self._window, center=self._center, min_periods=0)
        lag_mean = rolled.mean().reset_index().astype(np.float16)
        lag_max = rolled.max().reset_index().astype(np.float16)
        lag_min = rolled.min().reset_index().astype(np.float16)
        lag_std = rolled.std().reset_index().astype(np.float16)
        c_chars = '_c' if self._center== True else ''
        for col in cols:
            weather_df[f'{col}_rmean_{self._window}{c_chars}'] = lag_mean[col]
            if mode == '_all':
                weather_df[f'{col}_rmax_{self._window}{c_chars}'] = lag_max[col]
                weather_df[f'{col}_rmin_{self._window}{c_chars}'] = lag_min[col]
                weather_df[f'{col}_rstd_{self._window}{c_chars}'] = lag_std[col]
        del group_df, rolled
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
    
print(AddWeatherLags(72, True).transform(weather_train.head(20)))

NameError: name 'AddWeatherLags' is not defined

In [ ]:
weather_pipes = Pipeline(
    steps=[
        #('convertToDatetime', ConvertToDatetime()),
        ('interpolateTempMix', InterpolateTempMix()),
        ('imputeWeather', ImputeWeather()),
        ('fillMean',FillMean(['air_temperature','dew_temperature'
                              , 'precip_depth_1_hr', 'sea_level_pressure'])),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('relativeHumidity', RelativeHumidity()),
        ('addWeatherLags3', AddWeatherRolls(3)),
        #('addWeatherLags3C', AddWeatherLags(3, True)),
        ('addWeatherRolls72', AddWeatherRolls(72)),
        ('addWeatherRolls96', AddWeatherRolls(96)),
    ]
)

In [ ]:
print(weather_train)

In [ ]:
all = weather_train.append(weather_test, ignore_index=True)
all = weather_pipes.transform(all)
all = alignWeather(all)

all.sample(20, random_state=42)

In [ ]:
all.dtypes

In [ ]:
all.to_pickle(f'../input/ashrae-energy-prediction-pickles/weather_processed{mode}.pickle')


In [ ]:
all.dtypes
